In [23]:
import pandas as pd
import numpy as np

from glob import glob
from tqdm import tqdm
from scipy import interpolate

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, LSTM, GRU, AveragePooling1D, GlobalAveragePooling1D

### 데이터 전처리

In [24]:
w_list = sorted(glob('data/water_data/*.csv'))
w_list

['data/water_data\\data_2012.csv',
 'data/water_data\\data_2013.csv',
 'data/water_data\\data_2014.csv',
 'data/water_data\\data_2015.csv',
 'data/water_data\\data_2016.csv',
 'data/water_data\\data_2017.csv',
 'data/water_data\\data_2018.csv',
 'data/water_data\\data_2019.csv',
 'data/water_data\\data_2020.csv',
 'data/water_data\\data_2021.csv',
 'data/water_data\\data_2022.csv']

In [25]:
pd.read_csv(w_list[0]).shape

(26496, 15)

In [26]:
pd.read_csv(w_list[0]).head(4)

,ymdhm,swl,inf,sfw,ecpc,tototf,tide_level,wl_1018662,fw_1018662,wl_1018680,fw_1018680,wl_1018683,fw_1018683,wl_1019630,fw_1019630
0,2012-05-01 00:00,24.800,555.0,219.07,24.93,555.0,445.0,310.7,469.05,300.2,0.0,290.0,729.80,275.3,540.18
1,2012-05-01 00:10,24.794,464.6,218.86,25.15,562.9,449.0,314.7,498.00,300.2,0.0,290.0,731.48,275.3,540.18
2,2012-05-01 00:20,24.789,478.1,218.69,25.31,576.4,451.0,313.7,490.68,301.2,0.0,290.0,726.42,275.3,540.18
3,2012-05-01 00:30,24.789,464.8,218.69,25.31,563.1,452.0,311.7,476.21,301.2,0.0,290.0,726.42,276.3,552.17


In [27]:
train_data = []
train_label = []
num = 0

for i in w_list[:-1]:
    
    tmp = pd.read_csv(i)
    tmp = tmp.replace(" ", np.nan)
    tmp = tmp.interpolate(method = 'values')
    tmp = tmp.fillna(0)
    
    for j in tqdm(range(len(tmp)-432)):
        train_data.append(np.array(tmp.loc[j:j + 431, ["swl", "inf", "sfw", "ecpc",
                                                       "tototf", "tide_level",
                                                       "fw_1018662", "fw_1018680",
                                                       "fw_1018683", "fw_1019630"]]).astype(float))
        
        train_label.append(np.array(tmp.loc[j + 432, ["wl_1018662", "wl_1018680",
                                                      "wl_1018683", "wl_1019630"]]).astype(float))

100%|██████████| 26064/26064 [00:36<00:00, 712.41it/s]


In [28]:
train_data = np.array(train_data)
train_label = np.array(train_label)

print(train_data.shape)
print(train_label.shape)

(260640, 432, 10)
(260640, 4)


### 모델링 및 모델 학습

In [29]:
input_shape = (train_data[0].shape[0], train_data[0].shape[1])

model = Sequential()
model.add(GRU(256, input_shape=input_shape))
model.add(Dense(4, activation = 'relu'))

optimizer = tf.optimizers.RMSprop(0.001)

model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])

In [30]:
model.fit(train_data, train_label, epochs=10, batch_size=512)

Epoch 1/10
510/510 [==============================] - 4708s 9s/step - loss: 66020.8828 - mae: 245.4993
Epoch 2/10
510/510 [==============================] - 4924s 10s/step - loss: 20503.3828 - mae: 121.6022
Epoch 3/10
510/510 [==============================] - 4975s 10s/step - loss: 4216.8623 - mae: 31.9397
Epoch 4/10
510/510 [==============================] - 5006s 10s/step - loss: 1548.0138 - mae: 15.1116
Epoch 5/10
510/510 [==============================] - 5009s 10s/step - loss: 870.6493 - mae: 10.9528
Epoch 6/10
510/510 [==============================] - 5045s 10s/step - loss: 617.3361 - mae: 9.5969
Epoch 7/10
510/510 [==============================] - 5049s 10s/step - loss: 504.5198 - mae: 8.9551
Epoch 8/10
510/510 [==============================] - 5051s 10s/step - loss: 436.0015 - mae: 8.4134
Epoch 9/10
510/510 [==============================] - 5020s 10s/step - loss: 377.2599 - mae: 7.9303
Epoch 10/10
510/510 [==============================] - 4999s 10s/step - loss: 349.4088 -

### 추론 데이터셋 만들기

In [31]:
test_data = []
test_label = []

tmp = pd.read_csv(w_list[-1])
tmp = tmp.replace(" ", np.nan)
# 이전값을 사용
tmp = tmp.fillna(method = 'pad')
tmp = tmp.fillna(0)
    
#tmp.loc[:, ["wl_1018662", "wl_1018680", "wl_1018683", "wl_1019630"]] = tmp.loc[:, ["wl_1018662", "wl_1018680", "wl_1018683", "wl_1019630"]]*100
    
for j in tqdm(range(4032, len(tmp)-432)):
    test_data.append(np.array(tmp.loc[j:j + 431, ["swl", "inf", "sfw", "ecpc",
                                                    "tototf", "tide_level",
                                                    "fw_1018662", "fw_1018680",
                                                    "fw_1018683", "fw_1019630"]]).astype(float))
        
    test_label.append(np.array(tmp.loc[j + 432, ["wl_1018662", "wl_1018680",
                                                    "wl_1018683", "wl_1019630"]]).astype(float))

100%|██████████| 6912/6912 [00:06<00:00, 1051.41it/s]


In [32]:
test_data = np.array(test_data)
test_label = np.array(test_label)

print(test_data.shape)
print(test_label.shape)

(6912, 432, 10)
(6912, 4)


### 제출 파일 만들기

In [33]:
pred = model.predict(test_data)

In [ ]:
pred = pd.DataFrame(pred)

In [ ]:
sample_submission = pd.read_csv("data/sample_submission.csv")

sample_submission["wl_1018662"] = pred[0]
sample_submission["wl_1018680"] = pred[1]
sample_submission["wl_1018683"] = pred[2]
sample_submission["wl_1019630"] = pred[3]

In [ ]:
sample_submission.to_csv("save/submission.csv", index = False)